In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os 


In [8]:
pip install tensorflow-metal==0.6

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow-macos==2.10

  Using cached tensorflow_macos-2.11.0-cp39-cp39-macosx_12_0_arm64.whl (215.6 MB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
  Attempting uninstall: tensorflow-macos
    Found existing installation: tensorflow-macos 2.10.0
    Uninstalling tensorflow-macos-2.10.0:
      Successfully uninstalled tensorflow-macos-2.

In [2]:
import numpy as np
import tensorflow as tf

bfloat16 = tf.bfloat16.as_numpy_dtype

np.array([1.0, 2.0, 3.0], dtype=bfloat16)
# array([bfloat16(1), bfloat16(2), bfloat16(3)], dtype=bfloat16)

array([1, 2, 3], dtype=bfloat16)

In [4]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [5]:
filenames=os.listdir("train")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

2023-01-07 12:51:57.305430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-07 12:51:57.306189: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 126, 126, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 63, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 61, 61, 64)       2

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [9]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [10]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "train",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "train", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)

test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "test1",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


/Users/kaustubhmanitripathi_1/opt/anaconda3/envs/pygplates/lib/python3.9/site-packages/keras/preprocessing/image.py:1139: UserWarning: Found 20000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [11]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [12]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/10


/var/folders/dq/drbmpdws0hd78mzjt74jnb4c0000gn/T/ipykernel_6546/2528005629.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(
2023-01-07 12:52:21.761328: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-07 12:52:22.678124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1333/1333 [==============================] - ETA: 0s - loss: 0.7531 - accuracy: 0.6269

2023-01-07 12:55:22.347200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1333/1333 [==============================] - 194s 138ms/step - loss: 0.7531 - accuracy: 0.6269 - val_loss: 0.6329 - val_accuracy: 0.6675 - lr: 0.0010
Epoch 2/10
1333/1333 [==============================] - 188s 141ms/step - loss: 0.5495 - accuracy: 0.7268 - val_loss: 0.6003 - val_accuracy: 0.6847 - lr: 0.0010
Epoch 3/10
1333/1333 [==============================] - 172s 129ms/step - loss: 0.5030 - accuracy: 0.7595 - val_loss: 0.4688 - val_accuracy: 0.7828 - lr: 0.0010
Epoch 4/10
1333/1333 [==============================] - 164s 123ms/step - loss: 0.4604 - accuracy: 0.7872 - val_loss: 0.4631 - val_accuracy: 0.7856 - lr: 0.0010
Epoch 5/10
1333/1333 [==============================] - 161s 121ms/step - loss: 0.4337 - accuracy: 0.8042 - val_loss: 0.4033 - val_accuracy: 0.8236 - lr: 0.0010
Epoch 6/10
1333/1333 [==============================] - 161s 121ms/step - loss: 0.4119 - accuracy: 0.8157 - val_loss: 0.4518 - val_accuracy: 0.7972 - lr: 0.0010
Epoch 7/10
1333/1333 [=======================

In [13]:
model.save("model1_catsVSdogs_10epoch.h5")

In [14]:
test_filenames = os.listdir("/Users/kaustubhmanitripathi_1/Downloads/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [15]:
test_generator = train_datagen.flow_from_dataframe(test_df,
"test1",x_col='filename',y_col=None,
target_size=Image_Size,
class_mode=None,
batch_size=batch_size
)

Found 12500 validated image filenames.


In [16]:
results={
    0:'cat',
    1:'dog'
}
from PIL import Image
import numpy as np
im=Image.open("test1/2667.jpg")
im=im.resize(Image_Size)
im=np.expand_dims(im,axis=0)
im=np.array(im)
im=im/255
pred=np.argmax(model.predict([im])[0], axis=-1)
print(results[pred])

2023-01-07 13:21:32.729998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
cat
